<a href="https://colab.research.google.com/github/ptsurko/AIML-notebooks/blob/master/Sequence_tagging_with_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# https://www.depends-on-the-definition.com/guide-sequence-tagging-neural-networks-python/

TensorFlow 2.x selected.


In [2]:
import os
import zipfile

os.environ['KAGGLE_USERNAME'] = "pavelkey" # username from the json file
os.environ['KAGGLE_KEY'] = "2d38d31c6d5bdc84afc023ad38f79088" # key from the json file

!kaggle datasets download -d abhinavwalia95/entity-annotated-corpus # api copied from kaggle

zipref = zipfile.ZipFile('/content/entity-annotated-corpus.zip', 'r')
zipref.extractall()
zipref.close()

data = pd.read_csv("ner_dataset.csv", encoding="latin1")

 64% 17.0M/26.4M [00:00<00:00, 32.0MB/s]
100% 26.4M/26.4M [00:00<00:00, 45.0MB/s]


In [3]:
data = data.fillna(method="ffill")
data.tail(10)

,Sentence #,Word,POS,Tag
1048565,Sentence: 47958,impact,NN,O
1048566,Sentence: 47958,.,.,O
1048567,Sentence: 47959,Indian,JJ,B-gpe
1048568,Sentence: 47959,forces,NNS,O
1048569,Sentence: 47959,said,VBD,O
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [4]:
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words); n_words
tags = list(set(data["Tag"].values))
n_tags = len(tags); 

print('num words: ', n_words)
print('num tags: ', n_tags)

num words:  35179
num tags:  17


In [0]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
getter = SentenceGetter(data)
sent = getter.get_next()
sentences = getter.sentences

In [0]:
max_len = 50
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words - 1)

y = [[tag2idx[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [0]:
y = [to_categorical(i, num_classes=n_tags) for i in y]
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)

In [0]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

# input = Input(shape=(max_len,))
# model = Embedding(input_dim=n_words, output_dim=50, input_length=max_len)(input)
# model = Dropout(0.1)(model)
# model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
# out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)  # softmax output layer
# model = Model(input, out)

model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding(input_dim=n_words, output_dim=50, input_length=max_len),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=100, return_sequences=True, recurrent_dropout=0.1)),
  tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_tags, activation="softmax")),
])

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=5, validation_split=0.1, verbose=1)


Train on 38846 samples, validate on 4317 samples
Epoch 1/5
19840/38846 [==============>...............] - ETA: 1:36 - loss: 0.2008 - accuracy: 0.9511

In [0]:
hist = pd.DataFrame(history.history)
plt.figure(figsize=(12,12))
plt.plot(hist["acc"])
plt.plot(hist["val_acc"])
plt.show()

NameError: ignored